In [ ]:
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize, Normalizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from tqdm import tqdm
from IPython.display import Image, display_svg, SVG
import os
from fastai.metrics import RocAuc

In [62]:
valid_pct = 0.2

In [102]:
data = pd.read_csv("archive/Base.csv")
for file in os.listdir("archive")[1:]:
    new = pd.read_csv("archive/" + file)
    data = pd.concat([data, new])
data = shuffle(data)
data = data.drop(["x1," "x2", "month"], axis=1) # Drop "velocity_6h", "velocity_4w", 

In [109]:
procs = [Categorify] # Normalizer
dep_var = "fraud_bool"
cont,cat = cont_cat_split(data, 1, dep_var=dep_var)
valid = TabularPandas(data[0:int(len(data)*valid_pct)], procs, cat, cont, y_names=dep_var)
train = TabularPandas(data[len(valid):len(data)], procs, cat, cont, y_names=dep_var)

In [110]:
m = rf(train.xs, train.y)

In [ ]:
predictions = np.stack([t.predict(valid.xs) for t in m.estimators_]).mean(0)
targets = np.array(valid.y)

In [121]:
false_positive_rate = fp_rate(predictions, targets) * 100
accuracy = get_accuracy(predictions, targets) * 100

print("Accuracy: " + str(accuracy) + " %")
print("False positive rate (FPR): " + str(false_positive_rate) + " %")

Accuracy: 98.88825 %
False positive rate (FPR): 1.1133333333333333 %


In [117]:
def fp_rate(preds, targs):
    fps = 0
    for i in range_of(preds):
        if targs[i] == 1: fps += 1
    return fps / len(preds)
def get_accuracy(preds, targets):
    corrects = 0;
    for pred, target in list(zip(preds, targets)):
        if (abs(pred - target) < 0.5): corrects += 1
    return corrects / len(preds)
def rf(xs, y, n_estimators=40, max_samples=200_000,
       max_features=0.5, min_samples_leaf=5, **kwargs):
    return RandomForestRegressor(n_jobs=-1, n_estimators=n_estimators, max_samples=max_samples, 
                                 max_features=max_features, min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)